# 用人工智能查找香音的眼睛
我们从之前的例子可以看出，如果不屏蔽非眼睛区域，非眼睛区域的紫色也会被替换，我们也就没法使用这些图片来做视频了。所以接下来，我将会用人工智能缩小眼睛的范围。我们要用到anime-eyes-cascade.xml这个cascade。我们先打开那张换色不太好的图片kanon2.png

In [1]:
%matplotlib inline
import cv2 
import numpy as np
kanon=cv2.imread("kanon2.png")




我们打开训练好的eye cascade分类器，这个分类器eye_cascade的输入是一张图片，它会输出一组正方形(x,y,w,h)的数组eyes，代表着它查到的眼睛。

In [2]:
eye_cascade=cv2.CascadeClassifier("anime-eyes-cascade.xml")
eyes=eye_cascade.detectMultiScale(kanon, scaleFactor = 1.2,minNeighbors = 4)
print(eyes)

[[1212  298  122  122]
 [1034  311  106  106]]


我们给用rectangle函数画出查到的眼睛。

In [3]:
for (x,y,w,h) in eyes:
    cv2.rectangle(kanon,(x,y),(x+w,y+h),(33, 162, 234),5)
cv2.imshow("kanon",kanon)
cv2.waitKey(0)
cv2.destroyAllWindows()

然后我们只转化eyes区域内的像素

In [4]:
def kanon_kuku(input,convert_param):
    input=cv2.cvtColor(input,cv2.COLOR_BGR2HSV)
    lower=np.array([104,75,62])
    upper=np.array([160,238,251])
    mask=cv2.inRange(input,lower,upper).copy().astype("float")
    mask=mask/255
    notmask=1.0-mask
    input=input.copy().astype("float64")
    inputmask=np.zeros_like(input)
    inputmask[:,:,0]=input[:,:,0]*mask
    inputmask[:,:,1]=input[:,:,1]*mask 
    inputmask[:,:,2]=input[:,:,2]*mask
    inputnotmask=np.zeros_like(input)
    inputnotmask[:,:,0]=input[:,:,0]*notmask
    inputnotmask[:,:,1]=input[:,:,1]*notmask 
    inputnotmask[:,:,2]=input[:,:,2]*notmask
    inputmask[:,:,0]=((inputmask[:,:,0]-convert_param[0][0])*convert_param[1][0]+convert_param[2][0])*mask
    inputmask[:,:,1]=((inputmask[:,:,1]-convert_param[0][1])*convert_param[1][1]+convert_param[2][1])*mask
    inputmask[:,:,2]=((inputmask[:,:,2]-convert_param[0][2])*convert_param[1][2]+convert_param[2][2])*mask
    output=inputmask+inputnotmask
    return cv2.cvtColor(output.astype(np.uint8),cv2.COLOR_HSV2BGR)

convert_param=[[104,75,62],[1.31481481481,1.20552147239,0.687830687],[60,50,125]]

def process_boxes(boxes):
    if len(boxes)==0:
        return None
    else:
        new_box=[]
        for box in boxes:
            a=box[0]
            b=box[1]
            c=box[2]
            d=box[3]
            ai=int(a)
            bi=int(b)
            ci=int(c)
            di=int(d)
            new_box.append((ai,bi,ci,di))
        return new_box

for eye in eyes:
    x,y,w,h=eye
    eye_pixels=kanon[y:y+h,x:x+w,:]
    eye_pixels=kanon_kuku(eye_pixels,convert_param)
    kanon[y:y+h,x:x+w,:]=eye_pixels
    
cv2.imshow("kanon",kanon)
cv2.waitKey(0)
cv2.destroyAllWindows()